In [ ]:
import gdown
gdown.download('https://bit.ly/3q9SZix','20s_best_book.json',quiet=False) # github에서 다운로드

import pandas as pd
books_df = pd.read_json('20s_best_book.json') # json -> 데이터프레임
print(books_df.head()) # 초반 5개 출력

books = books_df[['no','ranking','bookname','authors','publisher','publication_year','isbn13']] # 해당 열만 뽑음
print(books.head())

print(books.loc[[0,1], ['bookname','authors']]) # or 슬라이딩 연산자['bookname':'authors'] 마지막 항목도 포함!!!
books = books_df.loc[:, 'no':'isbn13'] # :만 선택하면 전체 행이나 열
print(books.head())

print(books_df.loc[::2, 'no':'isbn13'].head()) # 스텝 지정 가능(2칸씩 건너뜀)

import requests
isbn = 9791190090018 # 우리기 빛의 속도로 갈 수 없다면의 isbn
url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
r = requests.get(url.format(isbn)) # 해당 isbn의 html 문서 얻기
#print(r.text)

from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser') # BeautifulSoup(파싱할 html, 파싱에 사용할 파서)
prd_link = soup.find('a',attrs = {'class':'gd_name'}) # 해당 html문서에서 a 태그 안의 클래스(속성) : 속성값인걸 찾기
print(prd_link) # <a class="gd_name" ...을 반환
print(prd_link['href']) # /Product/Goods/74261416 태그 안의 속성 참조

url = 'http://www.yes24.com'+prd_link['href']
r = requests.get(url)
#print(r.text)

soup = BeautifulSoup(r.text, 'html.parser')
prd_detail = soup.find('div',attrs={'id':'infoset_specific'}) # div 태그 안의 속성이 id : infoset_specific인걸 찾는다.
#print(prd_detail)

prd_tr_list = prd_detail.find_all('tr') # tr 태그 안의 내용 모두 가져오기
print(prd_tr_list)
for tr in prd_tr_list: # tr 태그 중에서
    if tr.find('th').get_text() =='쪽수, 무게, 크기': # tr 안의 th를 찾았는데 테그 안에 text 중 쪽수,무게,크기가 있다면 td안에 있는 내용을 page_td에 넣는다.
        page_td = tr.find('td').get_text()
        break
print()

def get_page_cnt(isbn): #isbn 입력하면 쪽수 반환
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 검색 결과에서 해당 도서를 선택합니다.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져옵니다.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 상품 상세정보 div를 선택합니다.
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    # 테이블에 있는 tr 태그를 가져옵니다.
    prd_tr_list = prd_detail.find_all('tr')
    # 쪽수가 들어 있는 th를 찾아 td에 담긴 값을 반환합니다.
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            return tr.find('td').get_text().split()[0]
    return ''

print(get_page_cnt(9791190090018))

top10_books = books.head(10) # 위의 10권만 가져옴

def get_page_cnt2(row):
    isbn = row['isbn13']
    return get_page_cnt(isbn)

page_count = top10_books.apply(get_page_cnt2, axis=1) # apply(실행할 함수,axis(행은 1,열은 0)) top10_books의 각각의 원소에 대해 get_page_cnt2를 실행함.
print(page_count) # 판다스 데이터

page_count.name = 'page_count'
print(page_count)

top10_with_page_count = pd.merge(top10_books, page_count,left_index=True, right_index=True) # merge(합칠 함수1,함수2,인덱스 기준으로 합침1,2)
print(top10_with_page_count)

Downloading...
From: https://bit.ly/3q9SZix
To: /content/20s_best_book.json
100%|██████████| 92.9k/92.9k [00:00<00:00, 40.6MB/s]


   no  ranking                     bookname   authors publisher  \
0   1        1  우리가 빛의 속도로 갈 수 없다면 :김초엽 소설   지은이: 김초엽        허블   
1   2        2         달러구트 꿈 백화점.이미예 장편소설   지은이: 이미예     팩토리나인   
2   3        3          지구에서 한아뿐 :정세랑 장편소설   지은이: 정세랑        난다   
3   4        4           시선으로부터, :정세랑 장편소설   지은이: 정세랑      문학동네   
4   5        5               아몬드 :손원평 장편소설   지은이: 손원평        창비   

  publication_year         isbn13 addition_symbol vol  class_no  \
0             2019  9791190090018           03810         813.7   
1             2020  9791165341909           03810         813.7   
2             2019  9791188862290           03810         813.7   
3             2020  9788954672214           03810         813.7   
4             2017  9788936434267           03810         813.7   

         class_nm  loan_count  \
0  문학 > 한국문학 > 소설         461   
1  문학 > 한국문학 > 소설         387   
2  문학 > 한국문학 > 소설         383   
3  문학 > 한국문학 > 소설         370   
4  문학 > 한국문학 > 소설         365